In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNet
from tensorflow.keras import layers, models

# Set the path to your dataset folder
dataset_folder = "Dataset"

# Define image size and batch size
img_size = (224, 224)
batch_size = 32

# Create data generators for training and validation
datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = datagen.flow_from_directory(
    dataset_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='training'
)

val_generator = datagen.flow_from_directory(
    dataset_folder,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    subset='validation'
)

# Load MobileNet model with pre-trained weights
base_model = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers of the pre-trained model
for layer in base_model.layers:
    layer.trainable = False

# Create a custom model on top of MobileNet
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(train_generator, epochs=10, validation_data=val_generator)

# Print the accuracy
accuracy = model.evaluate(val_generator)[1]
print(f"Accuracy on validation set: {accuracy}")


Found 6640 images belonging to 2 classes.
Found 1659 images belonging to 2 classes.
17225924/17225924 [==============================] - 2s 0us/step
Epoch 1/10
208/208 [==============================] - 210s 996ms/step - loss: 0.3416 - accuracy: 0.8575 - val_loss: 0.3648 - val_accuracy: 0.8704
Epoch 2/10
208/208 [==============================] - 187s 899ms/step - loss: 0.2341 - accuracy: 0.9033 - val_loss: 0.3698 - val_accuracy: 0.8620
Epoch 3/10
208/208 [==============================] - 200s 961ms/step - loss: 0.1957 - accuracy: 0.9252 - val_loss: 0.3734 - val_accuracy: 0.8620
Epoch 4/10
208/208 [==============================] - 188s 905ms/step - loss: 0.1721 - accuracy: 0.9367 - val_loss: 0.5132 - val_accuracy: 0.7607
Epoch 5/10
208/208 [==============================] - 191s 918ms/step - loss: 0.1567 - accuracy: 0.9437 - val_loss: 0.4346 - val_accuracy: 0.8143
Epoch 6/10
208/208 [==============================] - 189s 908ms/step - loss: 0.1412 - accuracy: 0.9532 - val_loss: 0.435

In [ ]:
# Save the model
model.save("Models\mobilenet_real_vs_spoof.h5")